In [129]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import os
import pandas as pd
from datetime import datetime
import numpy as np
import warnings
import platform

warnings.filterwarnings("ignore")

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None

In [130]:
random_state = 57
now = lambda: datetime.now().strftime("%Y-%m-%dT%Hh%Mm%Ss")


def is_databricks():
    return True if "DATABRICKS_RUNTIME_VERSION" in os.environ else False


running_on = platform.system()

if running_on == "Windows":
    path = fr"C:\Users\pmm\ownCloud\jupyter\22-05 HR"
elif running_on == "Darwin":
    path = f"/Users/pmm/ownCloud/jupyter/22-05 HR"
elif "google.colab" in str(get_ipython()):
    path = "/content"
elif is_databricks():
    my_container = "projects"
    my_lake = "tdsdsdatalake"
    mount_point = f"/mnt/{my_container}"
    path = f"{mount_point}"

    configs = {
        "fs.azure.account.auth.type": "CustomAccessToken",
        "fs.azure.account.custom.token.provider.class": spark.conf.get(
            "spark.databricks.passthrough.adls.gen2.tokenProviderClassName"
        ),
    }

    if not any(
        mount.mountPoint == mount_point for mount in dbutils.fs.mounts()
    ):
        dbutils.fs.mount(
            source=f"abfss://{my_container}@{my_lake}.dfs.core.windows.net/",
            mount_point=mount_point,
            extra_configs=configs,
        )

In [131]:
test_data = pd.read_parquet("test_data.parquet")
test_target = pd.read_csv("test_target.csv", index_col=0)
test_data = pd.concat([test_data, test_target], axis=1)

In [132]:
dtypes = {
    "pers_no": "object",
    "job_key": "object",
    "position": "object",
    "job_abbrev": "object",
    "contract": "object",
    "act": "object",
    "reason": "int64",
    "start_date": "datetime64[ns]",
    "most_recent_hire_date_01": "datetime64[ns]",
    "service_anniversary_16": "datetime64[ns]",
    "cocd": "object",
    "org_unit": "object",
    "employee_group": "object",
    "employee_subgroup": "object",
    "commission_type": "object",
    "pay_scale_group_effective_date": "datetime64[ns]",
    "ps_group": "object",
    "prev_ps_group": "object",
    "annl_sal_hrly_wage": "float64",
    "annual_salary": "float64",
    "pay_level_midpoint": "float64",
    "prev_wage": "float64",
    "pts": "int64",
    "supervisor": "object",
    "ps_area": "object",
    "aap": "object",
    "city": "object",
    "state": "object",
    "generation": "object",
    "retiree_medical_eligibility": "object",
    "functional_area": "object",
    "fst": "object",
    "age_at_action": "float64",
    "percent_of_market_after_action": "float64",
    "percent_of_market_before_action": "float64",
    "hired": "object",
    "term": "object",
    "retirement": "object",
    "promotion": "object",
    "progression": "object",
    "previous_job_code": "object",
    "current_bu": "object",
    "previous_bu": "object",
    "gender_most_current": "object",
    "race_ethinicity_most_current": "object",
    "poc_non_poc_most_current": "object",
    "tds_vet_excludes_v1_v9_most_current": "object",
    "disabled_most_current": "object",
    "tds_vet_excludes_v1_v9_most_current": "object",
}

dt = {col: dtypes[col] for col in test_data.columns if col in dtypes}
test_data = test_data.astype(dt)

In [133]:
df = test_data.copy()

In [134]:
df = df.reset_index().sort_values(by=["pers_no", "start_date"])

In [135]:
# keep only the most recent salary increase
df = df[(df["act"] == "ZD") & (df["reason"] == 8)].drop_duplicates(
    subset=["pers_no"]
)

In [136]:
# calculate previous sllary based on percen of market before and after
df["pct_chg"] = (
    df["percent_of_market_after_action"]
    - df["percent_of_market_before_action"]
)
df["previous_salary"] = df["annual_salary"] / (1 + df["pct_chg"])

In [137]:
# adjust salary only if increase was < 5%
df["annual_salary"] = np.where(
    (df["pct_chg"] < 0.05) & (df["percent_of_market_before_action"] > 0.1),
    (df["previous_salary"] * 0.95),
    df["annual_salary"],
)

In [138]:
df = df.set_index("index", drop=True)

test_data.loc[df.index, "annual_salary"] = df.loc[df.index, "annual_salary"]

In [139]:
test_data.to_parquet("test_data_95.parquet")